In [ ]:
'''

Este jupyter es un copy paste de otro que github no me deja subir por ser demasiado grande, es lo mismo pero sin correr 
las celdas, alguna información relevante para entender el proceso la pondré según vea. 

'''

In [ ]:
import pandas as pd
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager 
PATH = ChromeDriverManager().install()     

driver=webdriver.Chrome(PATH)      
from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys 
import time

import warnings
warnings.filterwarnings('ignore')
import regex as re

from bs4 import BeautifulSoup as bs 

In [ ]:
options = Options()
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")


In [ ]:
from bs4 import BeautifulSoup as bs
import requests as req
from pyshadow.main import Shadow

In [ ]:
html = req.get(URL)

html_content = html.text


soup=bs(html_content, 'html.parser')

soup

In [ ]:
URL = "https://archive.org/search?query=%28philosophy+%29&and%5B%5D=lending%3A%22is_readable%22&and%5B%5D=mediatype%3A%22texts%22&and%5B%5D=language%3A%22English%22"

In [ ]:
from joblib import Parallel, delayed

BASE_URL = "https://www.gutenberg.org/ebooks/search/?query=philosophy&submit_search=Go%21&start_index={}"

# Crear una lista con los índices de inicio para cada página
start_indices = [i for i in range(26, 526, 25)]

# Crear un conjunto de procesos en paralelo para extraer datos de todas las páginas

results2 = Parallel(n_jobs=6)(delayed(extract_gut)(BASE_URL.format(start_index)) for start_index in start_indices)


In [ ]:

def extract_gut(url):
    res = req.get(url)
    soup = bs(res.text, "html.parser")
    autores = []
    links_gut = []
    for link in soup.find_all("a", {"class": "link"}):
        l = link.get("href")
        if len(l) < 15:
            links_gut.append("https://www.gutenberg.org" + l)

    links_obras = []
    for enlace in links_gut:
        rec = req.get(enlace)
        sopa = bs(rec.text, "html.parser")
        for en in sopa.find_all("a", {"class": "link"}):
            l = en.get("href")
            if ".html.images" in l:
                try:
                    for name in sopa.find_all("h1"):
                        autor = name.text
                        autores.append(autor)
                        links_obras.append("https://www.gutenberg.org" + l)
                except:
                    pass

    obras_autores = {}
    for i, enla in enumerate(links_obras):
        rec = req.get(enla)
        sopa = bs(rec.text, "html.parser")
        temp = [p.text for p in sopa.find_all("p")]
        obras_autores[autores[i]] = temp

    return obras_autores

BASE_URL = 'https://www.gutenberg.org/ebooks/search/?query=philosophy&submit_search=Go%21&start_index={}'

# Crear una lista con los índices de inicio para cada página
start_indices = [i for i in range(26, 526, 25)]

# Crear un conjunto de procesos en paralelo para extraer datos de todas las páginas

results2 = Parallel(n_jobs=6)(delayed(extract_gut)(BASE_URL.format(start_index)) for start_index in start_indices)
df = pd.DataFrame(columns=["author", "title", "work"])


In [ ]:
df = pd.DataFrame(columns=["author", "title", "work"])

#en esta función limpiamos y separamos los libros enteros, quitamos las cosas raras que salgan de bajarlos y los ponemos
#en un df

def process_data(key, value):
    split = key.split(" by ")
    if len(split) >= 2:
        author = split[0]
        title = split[1]
        processed_data = []
        for s in value:
            s = s.replace('\r', '').replace('\n', '')  
            processed_data.append({"author": author, "title": title, "work": s})
        return processed_data
    else:
        print("Invalid format:", key)
        return []

In [ ]:
results = Parallel(n_jobs=6)(delayed(process_data)(k, v) for dic in results2 for k, v in dic.items())
df = pd.concat([df, pd.DataFrame([item for sublist in results for item in sublist])], ignore_index=True)


In [ ]:
df.to_csv(r"C:\Users\Juan\Desktop\placeholder_name\data.csv")

In [ ]:
#no todos los libros estaban en ingles
from langdetect import detect

def detect_language(text):
    try:
        lang = detect(text)
        return lang
    except:
        return None
merged_df['language'] = merged_df['work'].apply(detect_language)

english_df = merged_df[merged_df['language'] == 'en']

In [ ]:
#merged_df.shape, english_df.shape
#aquí reducía de 383 a 298 filas, asumible


In [ ]:
# quitamos la columna de lenguaje
#english_df = english_df.drop('language', axis=1)



In [ ]:
english_df.to_csv(r"C:\Users\Juan\Desktop\placeholder_name\dataen.csv")